## ARcode: HPC Application Recognition Through Image-encoded Monitoring Data

This Jupyter notebook contains the necessary code for building the ARcode model for application recognition.

### Loading libraries

In [ ]:
import numpy as np
import tensorflow as tf
from pathlib import Path

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, confusion_matrix

from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import model_from_json, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LeakyReLU
from tensorflow.keras.regularizers import l2

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### Loading dataset

In [ ]:
# The path to the dataset
DATA_PATH = Path('../dataset')

# Load labels and job signatures
labels_file_path = DATA_PATH / "labels.npy"
signatures_file_path = DATA_PATH / "signatures.npy"

labels = np.load(labels_file_path)
signatures = np.load(signatures_file_path)

# Mapping of IDs to application names. This mapping is used when creating the dataset.
app_code = {0: "BerkeleyGW", 1: "Espresso", 2: "Gromacs", 3: "LAMMPS", 4: "NWCHEM", 5: "VASP", 6: "WRF", 7: "aims", 8: "chroma", 9: "cp2k", 10: "e3sm", 11: "su3"}

# Get the input shape
input_shape = signatures[0].shape

# Labels and signatures shape
print(f'Labels shape: {labels.shape}')
print(f'Sigantures shape: {signatures.shape}')

In [ ]:
from timeit import default_timer as timer

class TimingCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

cb = TimingCallback()

### Splitting the dataset

In [ ]:
# Split the dataset to the training set (80%) and the testing set (20%).
X_train, X_test, y_train, y_test = train_test_split(signatures, labels, test_size=0.2, random_state=42)

# Change the labels from categorical to one-hot encoding
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

# Further split the training set (80%) to the training set (60%) and the validation set (20%).
X_train,X_valid,y_train,y_valid = train_test_split(X_train, y_train_one_hot, test_size=0.25, random_state=42)

print('Training set shape : ', X_train.shape, y_train.shape)
print('Validation set shape : ', X_valid.shape, y_valid.shape)
print('Testing set shape : ', X_test.shape, y_test.shape)

### Configuring the CNN Model

In [ ]:
batch_size = 32
epochs = 50
num_classes = 12

weight_initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)
bias_initializer=tf.keras.initializers.Zeros()

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape, padding='same', kernel_regularizer=l2(0.001), kernel_initializer=weight_initializer,bias_initializer=bias_initializer))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2, 2),padding='same'))

model.add(Conv2D(64, (3, 3), activation='relu',padding='same', kernel_regularizer=l2(0.001), kernel_initializer=weight_initializer,bias_initializer=bias_initializer))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

model.add(Conv2D(128, (3, 3), activation='relu',padding='same', kernel_regularizer=l2(0.001), kernel_initializer=weight_initializer,bias_initializer=bias_initializer))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))                  
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.001), kernel_initializer=weight_initializer,bias_initializer=bias_initializer))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.7))

model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])

model.summary()

### Training the Model

In [ ]:
signature_train = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_valid, y_valid), callbacks=[cb])

In [ ]:
print(cb.logs)
print(sum(cb.logs))

### Evaluation on the Testing Set

In [ ]:
test_eval = model.evaluate(X_test, y_test_one_hot, verbose=0)
print(f'Test loss: {test_eval[0]}')
print(f'Test accuracy: {test_eval[1]}')

In [ ]:
# Training and validation accuracy and loss
accuracy = signature_train.history['accuracy']
val_accuracy = signature_train.history['val_accuracy']
loss = signature_train.history['loss']
val_loss = signature_train.history['val_loss']
epochs = range(len(accuracy))

plt.plot(epochs, accuracy, '-', label='Training accuracy')
plt.plot(epochs, val_accuracy, '--', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, '-', label='Training loss')
plt.plot(epochs, val_loss, '--', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

### Classification Report

In [ ]:
# Predict application based on the threshold value; unknown application is marked as -1.
def threshold_prediction(prediction, threshold):
    updated_prediction = []
    for i in prediction:
        max_value = max(i)
        if max_value >= threshold:
            index = np.where(i == max_value)
            updated_prediction.append(index[0][0])
        else:
            updated_prediction.append(-1)
    return np.array(updated_prediction)

# Use 0.99 as the highest threshold value.
thresholds = [i for i in np.arange(0, 1.00, 0.05)]
thresholds.append(0.99)

In [ ]:
accuracy = []

predicted_classes = model.predict(X_test)

for threshold in thresholds:
    y_pred = threshold_prediction(predicted_classes, threshold)
    accu = accuracy_score(y_test, y_pred)
    accuracy.append(accu)

accuracy = [float("{:.4f}".format(s)) for s in accuracy]

print(f'Accuracy scores on different confidence thresholds: {accuracy}')

### Using Kfold to test the performance of the CNN model on identifying each application

In [ ]:
k_fold = KFold(n_splits=10, shuffle=True)

threshold = 0.8
accuracy_all = []

for train_idx, test_idx in k_fold.split(signatures, labels):
    X_train = signatures[train_idx]
    y_train = labels[train_idx]
    
    X_test = signatures[test_idx]
    y_test = labels[test_idx]
    
    # Change the labels from categorical to one-hot encoding
    y_train_one_hot = to_categorical(y_train)
    y_test_one_hot = to_categorical(y_test)
    
    predicted_classes = model.predict(X_test)
    y_pred = threshold_prediction(predicted_classes, threshold)
    
    # Get the accuracy scores
    matrix = confusion_matrix(y_test, y_pred)
    score = matrix.diagonal()/matrix.sum(axis=1)
    
    # Formatting the scores and ignore the score of unknow application
    score = [float("{:.4f}".format(s)) for s in score[1:]]
    
    print(score)
    
    accuracy_all.append(score)

### Saving the CNN model (optional)

In [ ]:
# # serialize model to JSON
# model_json = model.to_json()
# with open("arcode.json", "w") as json_file:
#     json_file.write(model_json)
    
# # serialize weights to HDF5
# model.save_weights("arcode.h5")
# print("Saved model to disk")

### Loading the CNN model (optional)

In [ ]:
# # load json and create model
# json_file = open('arcode.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()

# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("arcode.h5")
# print("Loaded model from disk")

# loaded_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])